# Reading the rest of HYDRUS files [Stage 2]
![The project files](../Assets/Project_files_2.png)

## Importing and loading dataset

In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

In [2]:
src = '../Datasets/H3D2_SandDitch0011'

## Unreadable files
Encrypted files that are converted to text elswhere
1. `h.out`
2. `Q.out`
3. `v.out`
3. `th.out`
4. `MESHTRIA.000`
4. `DOMAIN.IN`

## Reading previous implemented files
### Reading the _mesh files_
1. `H.TXT`
2. `V.TXT`
3. `TH.TXT`
4. `MESHTRIA.TXT`
### Spread-info files

1. `Cum_Q.out`
1. `h_Mean.out`
1. `v_Mean.out`
5. `Run_Inf.out`
### the _information files_
1. `A_Level.out`
2. `ATMOSPH.IN`
3. `Balance.out`
4. `DIMENSIO.IN`
5. `Run_Inf.out` (Repeated)
6. `SELECTOR.IN`

## Files to be implemented here


### Flux-info files

1. `Boundary.out`
1. `BOUNDARY.IN`
1. `Check.out`

In [3]:
# def read_hydrus_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs, 
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''
#     # Specify the source folder
#     if folder == 'Current':
#         read_dir = os.getcwd()
#     else:
#         read_dir = folder

#     # Finding number of nodes in the file
#     mesh_file = os.path.join(read_dir, 'MESHTRIA.TXT')
#     num_cells = np.array(linecache.getline(mesh_file, 6).split(), int)[0]
#     # Define dataframe titles
#     titles = ['n', 'x', 'y', 'z']
#     # Define a list of coordinates
#     full_data = [[0, 0, 0, 0]]
#     # Set a loop to geather all coordinates from MESHTRIA.TXT file
#     for i in range(8, num_cells + 8):
#         full_data.append(
#             np.array(linecache.getline(mesh_file, i).split(), float))
#     # Convert the list to numpy array then to a dataframe
#     coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)

#     # Print head and tail of the dataframe to ensure correctness
#     # pd.concat([coordinates_df.head(),coordinates_df.tail()])

#     # -----------------------------#
#     # To get data from all files   #
#     # -----------------------------#
#     def get_data_from_file(filename='TH.TXT', main_caption='Theta'):
#         '''
#         Function to combine all values of a property to a single dataframe 
#         inputs:
#         filename, the name of the file
#         caption, the leading caption of the columns (we will add the portion '_T= xxx')
#         where xxx is the timestep
#         '''
#         # compute number of lines for each timestep
#         num_lines = int(math.ceil(num_cells / 10.))
#         time_steps_remaining = True  # Flag to see if the loop should continue or not.
#         times_df = pd.DataFrame([])  # Empty dataframe
#         time_loc_start = 2  # The starting cell of the timestep
#         # Check if it is a velocity file
#         processing_velocity = (filename[-5:] == 'V.TXT')
#         while time_steps_remaining:
#             line_t = linecache.getline(filename, time_loc_start).split()
#             # Check if it is the start of the timestep, otherwise exit
#             if line_t[0] == 'Time':
#                 t = int(line_t[2])
#                 if processing_velocity:
#                     velocity_component = {
#                         'first': '1',
#                         'second': '2',
#                         'third': '3'
#                     }[line_t[5].strip()]
#                     caption = main_caption + velocity_component
#                 else:
#                     caption = main_caption
#                 # Finding the last line of the timestep
#                 tim_loc_end = num_lines + time_loc_start + 2
#                 # The starting time is always 0 because steps starts in 1 in HYDRUS
#                 time_data = [0]
#                 # Create the timestep as one long list
#                 for i in range(time_loc_start + 2, tim_loc_end):
#                     time_data.extend(linecache.getline(filename, i).split())
#                 # Convert the list to DataFrame
#                 dft = pd.DataFrame(
#                     np.array(time_data, float),
#                     columns=['{}_T{}'.format(caption, t)])
#                 if len(times_df) == 0:  # If it is the first timestep
#                     times_df = dft.copy()
#                 else:  # Otherwise (for all other timesteps)
#                     times_df = pd.concat([times_df, dft], axis=1)
#                 # Change the start to the probable next timestem (if exist)
#                 time_loc_start = tim_loc_end + 1
#                 time_steps_remaining = True if len(
#                     linecache.getline(filename, time_loc_start)) > 0 else False
#                 # End IF
#         return times_df

#     # Set the basic dataframe to the coordinates dataframe, to append to it.
#     full_df = coordinates_df
#     # Looping through the basic output files then to concatenate them all
#     properties = [('TH.TXT', 'Th'), ('H.TXT', 'H'), ('V.TXT', 'V')]
#     if not read_velocities:
#         properties = properties[:-1]
#     for prop in properties:
#         file_path = os.path.join(read_dir, prop[0])
#         # Check if the file exists
#         if os.path.isfile(file_path):
#             prop_df = get_data_from_file(file_path, prop[1])
#             full_df = pd.concat([full_df, prop_df], axis=1)
#         else:
#             print(
#                 'Warning, the file {} does not exist in the given path'.format(
#                     prop[0]))

#     # Convert the num column to integer
#     full_df[['n']] = full_df[['n']].astype(np.int64)
#     # dropping the first row (the zeros row) as it is not necessary
#     full_df.drop(0, inplace=True)
#     # Saving the resultant dataframe to disk.
#     if save_to_csv:
#         full_df.to_csv(os.path.join(read_dir, 'nesr_data2.csv'))
#     return full_df

## Reading the `BOUNDARY.IN` file

In [4]:
folder =src

# def read_boundary_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs, 
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''

# Specify the source folder
if folder == 'Current':
    read_dir = os.getcwd()
else:
    read_dir = folder



# Finding number of nodes in the file
mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
# num_cells = np.array(linecache.getline(mesh_file, 6).split(), int)#[0]
linecache.getline(mesh_file, 4).split()
# # Define dataframe titles
# titles = ['n', 'x', 'y', 'z']
# # Define a list of coordinates
# full_data = [[0, 0, 0, 0]]
# # Set a loop to geather all coordinates from MESHTRIA.TXT file
# for i in range(8, num_cells + 8):
#     full_data.append(
#         np.array(linecache.getline(mesh_file, i).split(), float))
# # Convert the list to numpy array then to a dataframe
# coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)

['518', '0', 'f', 't', 'f', 'f']

In [5]:
folder = src

# def read_boundary_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs,
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''

# Specify the source folder
if folder == 'Current':
    read_dir = os.getcwd()
else:
    read_dir = folder

# Finding number of nodes in the file
mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
num_cells = int(linecache.getline(mesh_file, 4).split()[0])
# Define dataframe titles
titles = ['n', 'surface_area']


def get_num_lines(num_cells, nums_per_line=10):
    num_lines = int(num_cells / nums_per_line)
    if num_cells % nums_per_line > 0: num_lines += 1
    return num_lines


def read_snakey_list(file_name, start_line, end_line, data_type=float):
    '''
    Reading a list of numbers thar are stored in a text file, where
    there are `numbers_count` numbers, stored as `nums_per_line` numbers 
    per line, the starting position of first occurence of numbers is at
    `start_line` line number
    returns a numpy array of the data with the `data_type` given
    '''
    points = []
    for i in range(start_line, end_line):
        points.extend(linecache.getline(file_name, i).split())
    return np.array(points, data_type)


num_lines = get_num_lines(num_cells, 10)

first_line = 10
end_line = first_line + num_lines
points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

first_line = end_line + 1
end_line = first_line + num_lines
areas = read_snakey_list(mesh_file, first_line, end_line)
data = np.array([points, areas]).T
data_in = pd.DataFrame(
    data, columns=titles).astype({
        "n": int,
        "surface_area": float
    })
data_in

,n,surface_area
0,1,0.020491
1,8,0.040982
2,9,0.762819
3,11,1.275030
4,12,0.512820
...,...,...
513,3152,1.538460
514,3153,1.538460
515,3154,1.538460
516,3155,1.538460


## Change the `get_means_table` function to read timed files

In [6]:
def get_means_table(filename,
                    header_location,
                    data_location,
                    units_location=None,
                    reading_end='end', 
                    replace_units=True):
    '''
    Returns a table of adjusted data to numeric analysis
    
    We have to specify the:
    header_location: the line number that contains the headers, 
    units_location : the line number that contains the units (if None, then
                        no units is allowed), 
    data_location  : the line number that contains the first line of data.
    
    if reading_end='end', it reads the lines until it finds the word 'end'
        (or any other word) at the beginining of the last line (it will 
        read before the 'end')
    if reading_end=integer, then it will read until it reaches the specified 
        number (inclusive)
        
    if replace_units=True, it will replace the units by equivalent numbers
        according to the following:
        'f', 't' >> 0, 1
        'mm', 'cm', 'm' >> 0, 1, 2
        'sec', 'min', 'hours', 'days', 'years' >> 0, 1, 2, 3, 4
        's', 'min', 'h', 'd', 'y' >>  0, 1, 2, 3, 4
    
    Units will be written after an underscore without any special chars
    e.g. velocity: m/s  --> velocity_mps
    
    # filename = os.path.join(file_path, 'Cum_Q.out')
    '''

    # Defining some functions
    def proper_type(x):
        try:
            nf = float(x)
            ni = float(int(nf))
            # print(nf, ni, abs(nf - ni))
            if abs(nf - ni) < 0.0000000000001:
                return int(ni)
            else:
                return nf
        except:
            return x

    def replace_text(x):
        if x in ('t', 'f'):
            # return {'t':1, 'f':0}[x]
            return ['f', 't'].index(x)
        elif x in ('mm', 'cm', 'm'):
            return ['mm', 'cm', 'm'].index(x)
        elif x in ('sec', 'min', 'hours', 'days', 'years'):
            return ['sec', 'min', 'hours', 'days', 'years'].index(x)
        elif x in ('s', 'min', 'h', 'd', 'y'):
            return ['s', 'min', 'h', 'd', 'y'].index(x)
        else:
            return x  # proper_type(x)

    def get_line(filename, pos, replace_units=True):
        line_feed = linecache.getline(filename, pos).split()
#         print(line_feed)
        if replace_units:
            return list(map(replace_text, line_feed))
        else:
            return line_feed

    def get_word(filename, pos, loc=0):
        word = get_line(pos)
        if len(word) < 1:
            return ''
        else:
            word = word[loc]
        if isinstance(word, str):
            return word.strip()
        else:
            return word

    def get_num(p1, p2, is2d):
        '''
        p1, the line of 2D file
        p2, the line of 3D file
        '''
        return {True: p1, False: p2}[is2d]

    def adjust_body(replaceable, headers, body):
        for _ in range(len(headers) - len(body)):
            body.append(replaceable)

    def reform_unit(unit, prefix=None):
        '''
        Functions to add units

        '''

        def split_letters(word):
            return [_ for _ in word]

        reformed = ''.join([{
            '/': 'p',
            '.': '',
            '[': '',
            ']': ''
        }.get(i, i) for i in split_letters(unit)])
        if prefix is None:
            return reformed
        else:
            return prefix + reformed

    # Defining variables
    headers = []
    body = []
    headers += get_line(filename, header_location, False)
#     print(headers)
    try:
        # If there is any empty columns, remove them
        for _ in range(5):
            headers.remove('...')
    except:
        pass
#     print(headers)
    headers = [_.replace('...', '') for _ in headers]

    if units_location is not None:
        units = get_line(filename, units_location, replace_units)
        units = [reform_unit(_, '_') for _ in units]
        headers = [x + y for x, y in zip(headers, units)]
    
    if type(reading_end)==str: # 'end' for example
        # reading to the end of the file
        i = data_location
        feed = get_line(filename, i, replace_units)
        while feed[0] != reading_end:
            body.append(feed)
            i += 1
            feed = get_line(filename, i, replace_units)
    else:
        for i in range (data_location, int(reading_end) + 1):
            body.append(get_line(filename, i, replace_units))

    body = np.array(body)
    return pd.DataFrame(body, columns=headers)

In [7]:
# test of previous implementation that it still works

get_means_table(os.path.join(src, 'Cum_Q.out'), 11, 14, 12).sample(3)

,Time_T,CumQAP_L3,CumQRP_L3,CumQA_L3,CumQR_L3,CumQ3_L3,CumQ1_L3,CumQS_L3,CumQ5_L3,CumQ6_L3,CumQ7_L3,CumQ8_L3,CumQ9_L3,CRunOff_L3,cEvapor_L3,cInfiltr_L3
203,8.8328,0.000E+00,0.000E+00,0.000E+00,0.000E+00,-0.353E+03,0.000E+00,0.000E+00,0.000E+00,0.152E-01,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00
56,0.5527,0.000E+00,0.000E+00,0.000E+00,0.000E+00,-0.221E+02,0.000E+00,0.000E+00,0.000E+00,0.332E-02,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00
192,7.8813,0.000E+00,0.000E+00,0.000E+00,0.000E+00,-0.315E+03,0.000E+00,0.000E+00,0.000E+00,0.145E-01,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00


In [8]:
# test of the new implementation that it works

get_means_table(os.path.join(src, 'Boundary.out'), 19, 22, units_location=None,
    reading_end=539,replace_units=False).sample(3)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
123,124,2762,2.16,1.54,21.10,-4,0.000E+00,0.000E+00,-25.42,0.088,20.000,0.000E+00
77,78,1024,0.00,3.08,0.00,-6,-0.788E-05,0.128E-03,-23.25,0.095,20.000,-0.107E-03
193,194,2832,11.15,4.62,21.46,-4,0.000E+00,0.000E+00,-27.04,0.084,20.000,0.000E+00


## Concatenating several dataframes for each timestep

In [9]:
df_t05 = get_means_table(
    os.path.join(src, 'Boundary.out'),
    19,
    22,
    units_location=None,
    reading_end=539,
    replace_units=False).sample(3)
df_t15 = get_means_table(
    os.path.join(src, 'Boundary.out'),
    544,
    547,
    units_location=None,
    reading_end=1064,
    replace_units=False).sample(3)

display(df_t05, df_t15)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
225,226,2864,11.15,6.15,21.46,-4,0.000E+00,0.000E+00,-27.67,0.083,20.000,0.000E+00
173,174,2812,29.14,3.08,22.18,-4,0.000E+00,0.000E+00,-25.96,0.087,20.000,0.000E+00
351,352,2990,8.15,12.31,21.34,-4,0.000E+00,0.000E+00,-27.80,0.082,20.000,0.000E+00


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
252,253,2891,3.66,7.69,21.16,-4,0.000E+00,0.000E+00,-32.68,0.074,20.000,0.000E+00
306,307,2945,36.63,9.23,22.48,-4,0.000E+00,0.000E+00,0.46,0.430,20.000,0.000E+00
434,435,3073,36.63,15.38,22.48,-4,0.000E+00,0.000E+00,0.50,0.430,20.000,0.000E+00


In [10]:
df_t05.assign(time_step= 5,inplace=True)#, df_t15['time_step']=15
display(df_t05, df_t15)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
225,226,2864,11.15,6.15,21.46,-4,0.000E+00,0.000E+00,-27.67,0.083,20.000,0.000E+00
173,174,2812,29.14,3.08,22.18,-4,0.000E+00,0.000E+00,-25.96,0.087,20.000,0.000E+00
351,352,2990,8.15,12.31,21.34,-4,0.000E+00,0.000E+00,-27.80,0.082,20.000,0.000E+00


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
252,253,2891,3.66,7.69,21.16,-4,0.000E+00,0.000E+00,-32.68,0.074,20.000,0.000E+00
306,307,2945,36.63,9.23,22.48,-4,0.000E+00,0.000E+00,0.46,0.430,20.000,0.000E+00
434,435,3073,36.63,15.38,22.48,-4,0.000E+00,0.000E+00,0.50,0.430,20.000,0.000E+00


In [11]:
df_t05 = df_t05.assign(time_step= 5)
df_t15 = df_t15.assign(time_step= 15)
display(df_t05, df_t15)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
225,226,2864,11.15,6.15,21.46,-4,0.000E+00,0.000E+00,-27.67,0.083,20.000,0.000E+00,5
173,174,2812,29.14,3.08,22.18,-4,0.000E+00,0.000E+00,-25.96,0.087,20.000,0.000E+00,5
351,352,2990,8.15,12.31,21.34,-4,0.000E+00,0.000E+00,-27.80,0.082,20.000,0.000E+00,5


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
252,253,2891,3.66,7.69,21.16,-4,0.000E+00,0.000E+00,-32.68,0.074,20.000,0.000E+00,15
306,307,2945,36.63,9.23,22.48,-4,0.000E+00,0.000E+00,0.46,0.430,20.000,0.000E+00,15
434,435,3073,36.63,15.38,22.48,-4,0.000E+00,0.000E+00,0.50,0.430,20.000,0.000E+00,15


In [12]:
pd.concat([df_t05, df_t15])

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
225,226,2864,11.15,6.15,21.46,-4,0.000E+00,0.000E+00,-27.67,0.083,20.000,0.000E+00,5
173,174,2812,29.14,3.08,22.18,-4,0.000E+00,0.000E+00,-25.96,0.087,20.000,0.000E+00,5
351,352,2990,8.15,12.31,21.34,-4,0.000E+00,0.000E+00,-27.80,0.082,20.000,0.000E+00,5
252,253,2891,3.66,7.69,21.16,-4,0.000E+00,0.000E+00,-32.68,0.074,20.000,0.000E+00,15
306,307,2945,36.63,9.23,22.48,-4,0.000E+00,0.000E+00,0.46,0.430,20.000,0.000E+00,15
434,435,3073,36.63,15.38,22.48,-4,0.000E+00,0.000E+00,0.50,0.430,20.000,0.000E+00,15


### Read the dataframe of the simulation

In [13]:
df = read_hydrus_data(folder=src, save_to_csv=False, read_velocities=True)
display(df.sample(3), get_full_simulation_info(df))

df_rotated = rotate_back(df, 2.2899, rotation_axis='y')
get_full_simulation_info(df_rotated)

 For the entire simulation
Time steps : [0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440] 
Dimensions : {'x': (-0.839105, 49.9601), 'y': (0.0, 20.0), 'z': (0.0, 22.9811)} 
Matrix dims: x_vals(41,), z_vals(46,), X(46, 41), Z(46, 41), M(46, 41)


 Variables statistics:


,n,x,y,z,Th_T0,Th_T5,Th_T15,Th_T30,Th_T60,Th_T120,...,V3_T240,V1_T500,V2_T500,V3_T500,V1_T720,V2_T720,V3_T720,V1_T1440,V2_T1440,V3_T1440
2286,2286,21.8028,9.23077,17.88550,0.15,0.222882,0.321478,0.386324,0.390000,0.360661,...,-0.000222,-0.000028,-1.209230e-07,-0.000109,-0.000026,-8.970330e-08,-0.000072,-0.000019,-5.376780e-08,-0.000027
2098,2098,32.6741,15.38460,21.32260,0.10,0.075774,0.430000,0.430000,0.430000,0.230872,...,-0.000884,-0.000302,-3.137600e-07,-0.000219,-0.000098,7.852460e-07,-0.000096,-0.000023,3.232820e-07,-0.000018
1608,1608,45.3837,18.46150,3.81646,0.15,0.203643,0.209360,0.211428,0.212564,0.365270,...,0.000889,0.000017,1.237330e-06,0.000322,0.000019,4.390130e-07,0.000158,0.000001,9.160260e-08,-0.000009


,Min,Mean,Median,Max
Th,0.049132,0.257289,3.117305e-01,0.430000
H,-676.912000,-66.645863,-1.962600e+01,12.429200
V1,-1.660930,-0.021988,-1.864930e-05,0.802749
V2,-0.263769,-0.000331,1.837060e-08,0.265775
V3,-1.086420,-0.002844,-2.684000e-05,0.777201


 For the entire simulation
Time steps : [0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440] 
Dimensions : {'x': (0.0, 50.0), 'y': (0.0, 20.0), 'z': (0.0, 21.0)} 
Matrix dims: x_vals(41,), z_vals(43,), X(43, 41), Z(43, 41), M(43, 41)


 Variables statistics:


,Min,Mean,Median,Max
Th,0.049132,0.257289,3.117305e-01,0.430000
H,-676.912000,-66.645863,-1.962600e+01,12.429200
V1,-1.660930,-0.021988,-1.864930e-05,0.802749
V2,-0.263769,-0.000331,1.837060e-08,0.265775
V3,-1.086420,-0.002844,-2.684000e-05,0.777201


In [14]:
get_available_timesteps(df)

[0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440]

**Pattern**
```
For T0005: 0017, 0019, 0022, 0539

For T0015: 0542, 0544, 0547, 1064

For T0030:   +3,   +2,   +3, +517

For T0030: 1067, 1069, 1072, 1589
```

In [15]:
titles_loc = 19
data_begins = 22
num_lines = 517
frames = []
for t in get_available_timesteps(df)[1:]:
    data_ends = data_begins + num_lines
    # print(titles_loc, data_begins, num_lines, data_ends)
    dft = get_means_table(
        os.path.join(src, 'Boundary.out'),
        titles_loc,
        data_begins,
        units_location=None,
        reading_end=data_ends,
        replace_units=False).sample(3)
    dft = dft.assign(time_step=t)
    frames.append(dft)
    titles_loc = data_ends + 5
    data_begins = titles_loc + 3
pd.concat(frames,ignore_index=True)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
0,31,152,27.64,0.00,22.12,-4,0.000E+00,0.000E+00,-26.51,0.085,20.000,0.000E+00,5
1,344,2982,44.12,10.77,22.78,-4,0.000E+00,0.000E+00,-2.41,0.425,20.000,0.000E+00,5
2,84,1030,0.00,7.69,0.00,-6,-0.923E-05,0.150E-03,-22.65,0.097,20.000,-0.114E-03,5
3,266,2904,23.14,7.69,21.94,-4,0.000E+00,0.000E+00,-8.50,0.254,20.000,0.000E+00,15
4,355,2993,12.65,12.31,21.52,-4,0.000E+00,0.000E+00,-31.28,0.076,20.000,0.000E+00,15
5,409,3047,45.62,13.85,22.84,-4,0.000E+00,0.000E+00,5.36,0.430,20.000,0.000E+00,15
6,372,3010,38.13,12.31,22.54,-4,0.000E+00,0.000E+00,2.20,0.430,20.000,0.000E+00,30
7,155,2793,0.66,3.08,21.04,-4,0.000E+00,0.000E+00,-37.46,0.068,20.000,0.000E+00,30
8,177,2815,33.63,3.08,22.36,-4,0.000E+00,0.000E+00,-0.24,0.430,20.000,0.000E+00,30
9,180,2818,38.13,3.08,22.54,-4,0.000E+00,0.000E+00,4.00,0.430,20.000,0.000E+00,60


## Adding the `surface area` to the above table

In [16]:
data_in.sample(3)

,n,surface_area
304,2943,2.30585
373,3012,2.30585
296,2935,2.30585


In [17]:
titles_loc = 19
data_begins = 22
num_lines = num_cells
frames = []
for t in get_available_timesteps(df)[1:]:
    data_ends = data_begins + num_lines -1
    print(titles_loc, data_begins, num_lines, data_ends)
    dft = get_means_table(
        os.path.join(src, 'Boundary.out'),
        titles_loc,
        data_begins,
        units_location=None,
        reading_end=data_ends,
        replace_units=False).sample(3)
    # add a column for time
    dft = dft.assign(time_step=t)
    # convert all columns to numeric
    cols = list(dft)
    dft[cols] = dft[cols].apply(pd.to_numeric, errors='coerce')
    dft = pd.merge(dft, data_in, on='n')
    frames.append(dft)
    titles_loc = data_ends + 5
    data_begins = titles_loc + 3
# display(frames)
pd.concat(frames,ignore_index=True)

19 22 518 539
544 547 518 1064
1069 1072 518 1589
1594 1597 518 2114
2119 2122 518 2639
2644 2647 518 3164
3169 3172 518 3689
3694 3697 518 4214
4219 4222 518 4739
4744 4747 518 5264


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,460,3098,26.14,16.92,22.06,-4,0.0,0.0,-28.79,0.080,20.0,0.0,5,2.30585
1,473,3111,45.62,16.92,22.84,-4,0.0,0.0,-1.46,0.430,20.0,0.0,5,2.30585
2,186,2824,47.12,3.08,22.90,-4,0.0,0.0,-1.00,0.430,20.0,0.0,5,2.69138
3,370,3008,35.13,12.31,22.42,-4,0.0,0.0,-0.30,0.430,20.0,0.0,15,2.30585
4,238,2876,29.14,6.15,22.18,-4,0.0,0.0,-3.34,0.407,20.0,0.0,15,2.30585
5,234,2872,23.14,6.15,21.94,-4,0.0,0.0,-8.43,0.256,20.0,0.0,15,2.30585
6,27,147,21.64,0.00,21.88,-4,0.0,0.0,-5.96,0.330,20.0,0.0,30,1.15292
7,502,3140,41.13,18.46,22.66,-4,0.0,0.0,3.88,0.430,20.0,0.0,30,2.30585
8,492,3130,26.14,18.46,22.06,-4,0.0,0.0,-4.14,0.387,20.0,0.0,30,2.30585
9,310,2948,41.13,9.23,22.66,-4,0.0,0.0,5.71,0.430,20.0,0.0,60,2.30585


## Creating a combined function for both files

### Creating a saving function

In [18]:
def save_this(dataframe, data_folder, output_name, authorized=True):
    """
    if the `authorized` flag is true, the `dataframe` will be saved as CSV 
        file, in the location `data_folder`//Nesr with the name `output_name`
    """

    
    if authorized:
        # check for the output folder
        save_folder = os.path.join(data_folder, 'Nesr')
        if not os.path.exists(save_folder):
            os.makedirs(save_folder) 
        # Saving
        filename = output_name
        if filename[-4:].lower() != '.csv':
            filename = f'{filename}.CSV'
        dataframe.to_csv(
            os.path.join(save_folder, filename), index=False)

In [19]:
save_this(df, src, 'Full_data', authorized=True)

In [20]:
'boundary_data.CSV'[-4:].lower() != '.csv'

False

### Create the combined function

In [21]:
def read_boundary_data(folder='Current',
                       titles_loc=19,
                       data_begins=22,
                       nums_per_line=10,
                       save_to_csv=True):
    '''
    A function to read both BOUNDARY.IN and  Boundary.outfiles from HYDRUS 
        outputs, then to:
            1- return one dataframe contains both data in a decent format.
            2- save this output to a CSV file (optional, True by default)
    Input:
        The name of the main folder (leave balank for the current folder)
        titles_loc=19, the line number of the first line contains captions
        data_begins=22, the line number of the first line contains data
        nums_per_line=10, teh number of columns per line in BOUNDARY.IN file
        The option to save_to csv, default =True (Boolean)
    '''

    # Specify the source folder
    if folder == 'Current':
        read_dir = os.getcwd()
    else:
        read_dir = folder

    # Finding number of nodes in the file
    mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
    num_cells = int(linecache.getline(mesh_file, 4).split()[0])
    # Define dataframe titles
    titles = ['n', 'surface_area']

    def get_num_lines(num_cells, nums_per_line=10):
        num_lines = int(num_cells / nums_per_line)
        if num_cells % nums_per_line > 0: num_lines += 1
        return num_lines

    def read_snakey_list(file_name, start_line, end_line, data_type=float):
        '''
        Reading a list of numbers thar are stored in a text file, where
        there are `numbers_count` numbers, stored as `nums_per_line` numbers 
        per line, the starting position of first occurence of numbers is at
        `start_line` line number
        returns a numpy array of the data with the `data_type` given
        '''
        points = []
        for i in range(start_line, end_line):
            points.extend(linecache.getline(file_name, i).split())
        return np.array(points, data_type)

    num_lines = get_num_lines(num_cells, nums_per_line)

    first_line = 10
    end_line = first_line + num_lines
    points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

    first_line = end_line + 1
    end_line = first_line + num_lines
    areas = read_snakey_list(mesh_file, first_line, end_line)
    data = np.array([points, areas]).T
    data_in = pd.DataFrame(
        data, columns=titles).astype({
            "n": int,
            "surface_area": float
        })

    # Adding the surface area to the above table
    # print(num_cells, nums_per_line, num_lines)
    frames = []
    for t in get_available_timesteps(df)[1:]:
        data_ends = data_begins + num_cells - 1
        # print(titles_loc, data_begins, num_lines, data_ends)
        dft = get_means_table(
            os.path.join(src, 'Boundary.out'),
            titles_loc,
            data_begins,
            units_location=None,
            reading_end=data_ends,
            replace_units=False)  #.sample(3)
        # add a column for time
        dft = dft.assign(time_step=t)
        # convert all columns to numeric
        cols = list(dft)
        # print(cols)
        # display(dft)
        dft[cols] = dft[cols].apply(pd.to_numeric, errors='coerce')
        dft = pd.merge(dft, data_in, on='n')
        frames.append(dft)
        titles_loc = data_ends + 5
        data_begins = titles_loc + 3
    # display(frames)
    frames = pd.concat(frames, ignore_index=True)
    # Saving if allowed

    #     if save_to_csv:
    #         # check for the output folder
    #         if not os.path.exists(os.path.join(src, 'Nesr')):
    #             os.makedirs(os.path.join(src, 'Nesr'))
    #         # Saving
    #         frames.to_csv(
    #             os.path.join(src, 'Nesr', 'boundary_data.CSV'), index=False)

    # save_this(dataframe, data_folder, output_name, authorized=True)
    save_this(frames, src, 'boundary_data.CSV', authorized=save_to_csv)
    return frames

In [22]:
read_boundary_data(src, 19, 22)
# read, displayed, and saved to the Nesr sub-folder

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,1,1,0.00,0.00,0.00,-6,-9.890000e-09,4.830000e-07,-57.69,0.056,20.0,-1.810000e-06,5,0.020491
1,2,8,-0.08,0.00,2.00,-6,-1.500000e-08,3.660000e-07,-164.11,0.198,20.0,-4.710000e-08,5,0.040982
2,3,9,-0.80,0.00,19.98,-6,-7.430000e-05,9.740000e-05,-30.62,0.077,20.0,-8.150000e-04,5,0.762819
3,4,11,-0.84,0.00,20.98,-4,0.000000e+00,0.000000e+00,-27.31,0.083,20.0,0.000000e+00,5,1.275030
4,5,12,49.12,0.00,22.98,-3,1.030000e+00,-2.000000e+00,0.48,0.430,20.0,5.130000e+00,5,0.512820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5175,514,3152,49.12,7.69,22.98,-3,0.000000e+00,0.000000e+00,-25.88,0.087,20.0,1.850000e+02,1440,1.538460
5176,515,3153,49.12,6.15,22.98,-3,0.000000e+00,0.000000e+00,-25.87,0.087,20.0,1.850000e+02,1440,1.538460
5177,516,3154,49.12,4.62,22.98,-3,0.000000e+00,0.000000e+00,-25.87,0.087,20.0,1.850000e+02,1440,1.538460
5178,517,3155,49.12,3.08,22.98,-3,0.000000e+00,0.000000e+00,-25.87,0.087,20.0,1.850000e+02,1440,1.538460


# Updating the `get_one_line_df` to add boundary areas from `Boundary.out` file

In [23]:
get_one_line_df(src, simulation_name="Sand Ditch simulation", dims='3d').T

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,hMean5,hMean15,hMean30,hMean60,hMean120,hMean180,hMean240,hMean500,hMean720,hMean1440
Sand Ditch simulation,80.78,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,-123.71,-112.81,-98.652,-67.802,-45.578,-37.625,-32.564,-23.068,-20.772,-19.604


In [24]:
# Get a one-column dataframe
dfg = get_one_line_df(
    src, simulation_name="Sand Ditch simulation", dims='3d')
dfg.shape

(139, 1)

In [25]:
filename = os.path.join(src, 'Boundary.out')
to_add = {}
for i in [12, 13, 14]:
    l=get_line(filename, i, replace_units=False)
    caption= ' '.join(l[:-1])
    data = proper_type(l[-1])
    to_add[caption]=data
to_add
    

{'Variable head or flux 1 boundary:': 19.99998,
 'Atmospheric boundary:': 990.2325,
 'Free or deep drainage boundary:': 12.09827}

In [26]:
brief_dictionary = {
    'Variable head or flux 1 boundary:': 'VarFluxArea',
    'Atmospheric boundary:': 'AtmosArea',
    'Free or deep drainage boundary:': 'DrainageArea'
}
# Replace names
keys = list(to_add.keys())
for key in keys:
    to_add[brief_dictionary[key]]=to_add[key]
    del to_add[key]
# to_add = to_add2
to_add

{'VarFluxArea': 19.99998, 'AtmosArea': 990.2325, 'DrainageArea': 12.09827}

In [27]:
# Convert the dictionary to a dataframe
pd.DataFrame.from_dict(to_add, orient='index', columns=dfg.columns)

,Sand Ditch simulation
VarFluxArea,19.99998
AtmosArea,990.23250
DrainageArea,12.09827


In [28]:
dfg = dfg.append(pd.DataFrame.from_dict(to_add, orient='index', columns=dfg.columns))
dfg

,Sand Ditch simulation
SimulTime_s,80.78000
L_Unit,1.00000
T_Unit,1.00000
Category,3.00000
MaxIt,10.00000
...,...
hMean720,-20.77200
hMean1440,-19.60400
VarFluxArea,19.99998
AtmosArea,990.23250


In [29]:
def save_this(dataframe,
              data_folder,
              output_name,
              authorized=True,
              save_type='csv',
              save_index=False):
    """
    if the `authorized` flag is true, the `dataframe` will be saved as 
        CSV or json files (depending on the `save_type`), 
        in the location `data_folder`//Nesr with the name `output_name`
    """

    if authorized:
        # check for the output folder
        save_folder = os.path.join(data_folder, 'Nesr')
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        # Saving
        filename = output_name
        if filename[-3:].lower() != save_type.lower():
            filename = f'{filename}.{save_type.lower()}'
        if save_type.lower() == 'csv':
            dataframe.to_csv(
                os.path.join(save_folder, filename), index=save_index)
        elif save_type.lower() == 'json':
            dataframe.to_json(os.path.join(save_folder, filename))

In [30]:
save_this(
    dfg,
    src,
    'Simulation_info',
    authorized=True,
    save_type='csv',
    save_index=True)

In [31]:
save_this(dfg, src, 'Simulation_info', authorized=True, save_type='json')

ValueError: DataFrame index must be unique for orient='columns'.
### The error shows that there are repeated index values in the dfg dataframe

In [32]:
# from itertools import groupby
# groupby(dfg.index.tolist())


# to find the repeated indeces
import collections
dfg_index = dfg.index.tolist()
display(collections.Counter(dfg_index).most_common(5))
print(dfg_index)

[('SimulTime_s', 1),
 ('L_Unit', 1),
 ('T_Unit', 1),
 ('Category', 1),
 ('MaxIt', 1)]

['SimulTime_s', 'L_Unit', 'T_Unit', 'Category', 'MaxIt', 'TolTh', 'TolH', 'InitH/W', 'lWat', 'lChem', 'lSink', 'Short', 'Inter', 'lScrn', 'AtmIn', 'lTemp', 'lWTDep', 'lEquil', 'lExtGen', 'lInv', 'lUnsatCh', 'lCFSTr', 'lHP2', 'm_lActRSU', 'lDummy', 'NMat', 'NLay', 'hTab1', 'hTabN', 'NAniz', 'Model', 'Hysteresis', 'thr', 'ths', 'Alfa', 'n', 'Ks', 'l', 'dt', 'dtMin', 'dtMax', 'DMul', 'DMul2', 'ItMin', 'ItMax', 'MPL', 'tInit', 'tMax', 'NumNPD', 'NumElD', 'NumBPD', 'MBandD', 'NSeepD', 'NumSPD', 'NDrD', 'NElDrD', 'NMatD', 'NObsD', 'NSD', 'NAnis', 'TLevel_i', 'Time_i', 'dt_i', 'Iter_i', 'ItCum_i', 'TLevel_e', 'Time_e', 'dt_e', 'Iter_e', 'ItCum_e', 'TimeBC0', 'CumQ3BC0', 'hAtmBC0', 'hKode3BC0', 'A-levelBC0', 'hCritABC0', 'Flux_rtBC0', 'TimeBC1', 'CumQ3BC1', 'hAtmBC1', 'hKode3BC1', 'A-levelBC1', 'hCritABC1', 'Flux_rtBC1', 'InFlow0', 'InFlow5', 'InFlow15', 'InFlow30', 'InFlow60', 'InFlow120', 'InFlow180', 'InFlow240', 'InFlow500', 'InFlow720', 'InFlow1440', 'VolumeW0', 'VolumeW5', 'VolumeW15', '

#### Let's review where is the error

In [33]:
folder_path=src
simulation_name="Nesr simulation"
dims='3d'
# def get_one_line_df(folder_path, simulation_name="Nesr simulation", dims='2d'):
    
# '''
# Combines the most important info from the following files into one dataframe:
#     `SELECTOR.IN`, `DIMENSIO.IN`, `Run_Inf.out`, `Balance.out`, `A_Level.out` and `ATMOSPH.IN`

# '''
# # Get the basic parameters
df_basic = read_selector_in(folder_path, dims)
df_basic
# Get the boundary conditions parameters
df_bcs = read_a_level_out(folder_path, dims).T
df_bcs
# converting it to one row
hds0 = ['Time', 'CumQ3', 'hAtm', 'hKode3', 'A-level', 'hCritA', 'rt']
hdsB = ['Time', 'CumQ3', 'hAtm', 'hKode3', 'A-level', 'hCritA', 'Flux_rt']
hds = []
vals = []
for col in df_bcs.columns:
    hds += list(map(lambda x: x + 'BC{}'.format(col), hdsB))
    for idx in hds0:
        if idx == 'rt':
            vals.append(df_bcs.loc[idx, col].iloc[0])
        else:
            vals.append(df_bcs.loc[idx, col])
df_bcs = pd.DataFrame(data=vals, index=hds).T
df_bcs
# Get the mass balance parameters
df_bal = read_balance_out(folder_path)[1]
hds0 = ['InFlow', 'VolumeW', 'WatBalR', 'WatBalT', 'hMean']
hds = []
vals = []
for col in hds0:
    for idx in df_bal.index:
        hds.append(col + str(int(df_bal.loc[idx, 'Time'])))
        vals.append(df_bal.loc[idx, col])
df_bal = pd.DataFrame(data=vals, index=hds).T
df_bal
# concatenate the 3 dfs
frames = [df_basic, df_bcs, df_bal]

df_result = pd.concat(frames, axis=1)
# df_result.columns
# df_result.rename({'0':"Custom Name"}, axis='columns')
df_result
df_result = df_result.rename({0: simulation_name}, axis='index')
df_result

# return df_result.T

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,hMean5,hMean15,hMean30,hMean60,hMean120,hMean180,hMean240,hMean500,hMean720,hMean1440
Nesr simulation,80.78,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,-123.71,-112.81,-98.652,-67.802,-45.578,-37.625,-32.564,-23.068,-20.772,-19.604


In [34]:
display(
    collections.Counter(df_basic.T.index.tolist()).most_common(4),
    collections.Counter(df_bcs.T.index.tolist()).most_common(4),
    collections.Counter(df_bal.T.index.tolist()).most_common(4),
    collections.Counter(df_result.T.index.tolist()).most_common(4))

[('SimulTime_s', 1), ('L_Unit', 1), ('T_Unit', 1), ('Category', 1)]

[('TimeBC0', 1), ('CumQ3BC0', 1), ('hAtmBC0', 1), ('hKode3BC0', 1)]

[('InFlow0', 1), ('InFlow5', 1), ('InFlow15', 1), ('InFlow30', 1)]

[('SimulTime_s', 1), ('L_Unit', 1), ('T_Unit', 1), ('Category', 1)]

#### The problem is in `df_basic` dataframe, i.e. in the `read_selector_in` function

In [35]:
file_path = src
geom = '3d'
# def read_selector_in(file_path, geom='2D'):
# '''
# Reading four files: `SELECTOR.IN`, `DIMENSIO.IN`, `Run_Inf.out`, `Balance.out`
# Returns a dataframe table of a summary of all infos in them
# '''

if geom.lower() == '2d':
    is2d = True
    start = 3
else:
    is2d = False
    start = 11
filename = os.path.join(file_path, 'SELECTOR.IN')
headers = ['L_Unit', 'T_Unit', 'Category']
categ = {
    0: 'Horizontal plane XY',
    1: 'Axisymmetric Vertical Flow',
    2: 'Vertical Plane XZ',
    3: '3D General Domain'
}
body = []


def proper_type(x):
    try:
        nf = float(x)
        ni = float(int(nf))
        # print(nf, ni, abs(nf - ni))
        if abs(nf - ni) < 0.0000000000001:
            return int(ni)
        else:
            return nf
    except:
        return x


def replace_text(x):
    if x in ('t', 'f'):
        # return {'t':1, 'f':0}[x]
        return ['f', 't'].index(x)
    elif x in ('mm', 'cm', 'm'):
        return ['mm', 'cm', 'm'].index(x)
    elif x in ('sec', 'min', 'hours', 'days', 'years'):
        return ['sec', 'min', 'hours', 'days', 'years'].index(x)
    elif x in ('s', 'min', 'h', 'd', 'y'):
        return ['s', 'min', 'h', 'd', 'y'].index(x)
    else:
        return x  # proper_type(x)


def get_line(pos):
    line_feed = linecache.getline(filename, pos).split()
    return list(map(replace_text, line_feed))


def get_word(pos, loc=0):
    word = get_line(pos)
    if len(word) < 1:
        return ''
    else:
        word = word[loc]
    if isinstance(word, str):
        return word.strip()
    else:
        return word


def get_num(p1, p2):
    '''
    p1, the line of 2D file
    p2, the line of 3D file
    '''
    return {True: p1, False: p2}[is2d]


def adjust_body(replaceable):
    for _ in range(len(headers) - len(body)):
        body.append(replaceable)


body.append(get_word(6))
body.append(get_word(7))

body.append({True: int(get_word(10)), False: 3}[is2d])
headers += get_line(get_num(11, 9))[:4]
body += get_line(get_num(12, 10))[:4]

headers += get_line(get_num(13, 11))
body += get_line(get_num(14, 12))
headers += get_line(get_num(15, 13))
body += get_line(get_num(16, 14))

headers += get_line(get_num(20, 18))
body += get_line(get_num(21, 19))
adjust_body(0)

headers += get_line(get_num(22, 20))
body += get_line(get_num(23, 21))
num_material = proper_type(body[-2])
print(num_material, type(num_material))

headers += get_line(get_num(24, 22))
body += get_line(get_num(25, 23))

# The location of dt, dMin, ... depends on the number of materials

headers += get_line(27 + num_material - 1)
body += get_line(28 + num_material - 1)
headers += get_line(29 + num_material - 1)
body += get_line(30 + num_material - 1)

# Getting data from the DIMENSIO.IN file
filename = os.path.join(file_path, 'DIMENSIO.IN')
headers += get_line(2)
body += get_line(3)
adjust_body(0)

# Getting data from the Run_Inf.out file
filename = os.path.join(file_path, 'Run_Inf.out')
# We want only the first and last entries of the table.

# Here is the first entry
headers += ['TLevel_i', 'Time_i', 'dt_i', 'Iter_i', 'ItCum_i']
body += get_line(5)
i = 6
# Bypassing the other rows
while get_word(i) != 'end':
    i += 1
#         print(i, get_word(i), end='||')
# Here is the last entry
headers += ['TLevel_e', 'Time_e', 'dt_e', 'Iter_e', 'ItCum_e']
body += get_line(i - 1)

# Getting data from the Balance.out file
filename = os.path.join(file_path, 'Balance.out')
headers = ['SimulTime_s'] + headers
i = 10
while get_word(i) != 'Calculation':
    i += 1
body = [get_word(i, loc=3)] + body

# finalize
body = np.array(body)
headers = np.array(headers)

df3 = pd.DataFrame(data=body, index=headers).T
df3= df3.apply(pd.to_numeric, errors='ignore')
df3 = (df3.T.loc[~df3.T.index.duplicated(keep='first')]).T
# return df
print(headers)
print(body)
df3

3 <class 'int'>
['SimulTime_s' 'L_Unit' 'T_Unit' 'Category' 'MaxIt' 'TolTh' 'TolH'
 'InitH/W' 'lWat' 'lChem' 'lSink' 'Short' 'Inter' 'lScrn' 'AtmIn' 'lTemp'
 'lWTDep' 'lEquil' 'lExtGen' 'lInv' 'lUnsatCh' 'lCFSTr' 'lHP2' 'm_lActRSU'
 'lDummy' 'lDummy' 'lDummy' 'NMat' 'NLay' 'hTab1' 'hTabN' 'NAniz' 'Model'
 'Hysteresis' 'thr' 'ths' 'Alfa' 'n' 'Ks' 'l' 'dt' 'dtMin' 'dtMax' 'DMul'
 'DMul2' 'ItMin' 'ItMax' 'MPL' 'tInit' 'tMax' 'NumNPD' 'NumElD' 'NumBPD'
 'MBandD' 'NSeepD' 'NumSPD' 'NDrD' 'NElDrD' 'NMatD' 'NObsD' 'NSD' 'NAnis'
 'TLevel_i' 'Time_i' 'dt_i' 'Iter_i' 'ItCum_i' 'TLevel_e' 'Time_e' 'dt_e'
 'Iter_e' 'ItCum_e']
['80.7799999713898' '1' '1' '3' '10' '0.001' '1' '1' '1' '0' '0' '0' '0'
 '1' '1' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '3' '1' '10000'
 '10000' '1' '3' '0' '0.045' '0.43' '0.145' '2.68' '0.495' '0.5' '0.001'
 '1e-005' '7200' '1.3' '0.7' '3' '7' '10' '0' '1440' '8250' '29738' '518'
 '30' '1' '1' '1' '20' '3' '1' '1' '1' '1' '0.10000000E-02' '0.10000E-02'
 '2' '2' '5

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,TLevel_i,Time_i,dt_i,Iter_i,ItCum_i,TLevel_e,Time_e,dt_e,Iter_e,ItCum_e
0,80.78,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,1.0,0.001,0.001,2.0,2.0,565.0,1440.0,114.75,3.0,2214.0


In [36]:
collections.Counter(df3.T.index.tolist()).most_common(4)

[('SimulTime_s', 1), ('L_Unit', 1), ('T_Unit', 1), ('Category', 1)]

# FIXED بفضل الله